In [1]:
from os import path
from collections import deque

from ga4stpg.graph import UGraph
from ga4stpg.graph.disjointsets import DisjointSets
from ga4stpg.graph.priorityqueue import PriorityQueue
from ga4stpg.graph.reader import read_problem
from ga4stpg.graph.util import has_cycle, how_many_components
from ga4stpg.util import STEIN_B

from ga4stpg.tree.evaluation import EvaluateTreeGraph
from ga4stpg.tree.generate import GenerateBasedPrimRST

In [2]:
INDEX_SELECT = 4

datasets = path.join('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary')

filename = path.join(datasets, STEIN_B[INDEX_SELECT][0])

print(filename)

STPG = read_problem(filename)

..\..\ppgi-stpg-gpx\datasets\ORLibrary\steinb5.txt


In [3]:
from ga4stpg.edgeset import UEdge, EdgeSet

class Segment:

    def __init__(self):
        self.edges = EdgeSet()
        self.cost = 0
        self.portals = set()

    def __len__(self):
        return len(self.edges)

    def __str__(self):
        return f'Segment <{len(self.edges)}>'

    def __iter__(self):
        return iter(self.edges)

    @property
    def bounds(self):
        return frozenset(self.portals)
    
    def add(self, v, u):
        self.edges.add(UEdge(v, u))
    

In [4]:
from random import randrange

def f_weight(v, u):
    if STPG.graph.has_edge(v, u):
        return STPG.graph.weight(v, u)
    else:
        return 0

def find_segments(graph, common_nodes):
    visited = set()

    # start = None
    # index = randrange(0, len(common_nodes))
    # for i, nro in enumerate(common_nodes):
    #     if i == index:
    #         start = nro
    #         break 
    # assert start is not None
    # stack_outter = [start]
    stack_outter = list(common_nodes)
    result = list()

    def search(start, neighbor):
        segment = Segment()
        segment.portals.add(start)
        segment.add(start, neighbor)
        segment.cost += f_weight(start, neighbor)
        
        stack_inner = [neighbor]

        while stack_inner:
            u = stack_inner.pop()
            visited.add(u)
            if u not in common_nodes:
                counter = 0
                for w in graph.adjacent_to(u):
                    if w not in visited:
                        stack_inner.append(w)
                        segment.add(u, w)
                        segment.cost += f_weight(u, w)
                        counter += 1
                if counter == 0:
                    segment.portals.add(u)                    
            else:
                stack_outter.append(u)
                segment.portals.add(u)
        # end while
        return segment
        # end search

    while stack_outter:
        s = stack_outter.pop()
        
        visited.add(s)
        for v in graph.adjacent_to(s):
            if v not in visited:
                seg = search(s, v)
                result.append(seg)

    return result

In [5]:
def check_portals(portals, disjoint):
    '''Verifica se os vértices portais de um segmento
    se conectam à mesma partição de vértices comuns. 

    Faz essa verificação em tempo O(n)
    '''
    f_check = set()

    for p in portals:
        if p not in disjoint:
            return False
        k = disjoint.find(p)
        if k in f_check:
            return False
        f_check.add(k)

    return True

In [6]:
edges = [
    (1, 3),
    (1, 15),
    (15, 17),
    (3, 5),
    (5, 11),
    (11, 13),
    (5, 7),
    (7, 9),
    (7, 23),
]

red = UGraph()

for v, u in edges:
    red.add_edge(v, u)

In [7]:
edges = [
    (1, 17),
    (1, 14),
    (14, 18),
    (18, 5),
    (5, 23),
    (23, 13),
    (5, 9)
]

blue = UGraph()

for v, u in edges:
    blue.add_edge(v, u)

In [8]:
common_nodes = set(red.vertices) & set(blue.vertices)

common_nodes

{1, 5, 9, 13, 17, 23}

In [9]:
red_segments  = find_segments(red, common_nodes)
blue_segments = find_segments(blue, common_nodes)

In [10]:
print("..:: RED SEGMENTS ::..", end='\n\n')
for seg in red_segments:
    print(seg, seg.portals)
    for edge in seg:
        print(edge, end="  ")
    print("\n\n")

..:: RED SEGMENTS ::..

Segment <3> {9, 5, 23}
Edge <(5, 7)>  Edge <(7, 23)>  Edge <(7, 9)>  


Segment <2> {13, 5}
Edge <(11, 13)>  Edge <(5, 11)>  


Segment <2> {1, 5}
Edge <(1, 3)>  Edge <(3, 5)>  


Segment <2> {1, 17}
Edge <(15, 17)>  Edge <(1, 15)>  




In [11]:
print("..:: BLUE SEGMENTS ::..", end='\n\n')

for seg in blue_segments:
    print(seg, seg.portals)
    for edge in seg:
        print(edge, end="  ")
    print("\n\n")

..:: BLUE SEGMENTS ::..

Segment <1> {13, 23}
Edge <(13, 23)>  


Segment <1> {5, 23}
Edge <(5, 23)>  


Segment <1> {9, 5}
Edge <(5, 9)>  


Segment <3> {1, 5}
Edge <(14, 18)>  Edge <(5, 18)>  Edge <(1, 14)>  


Segment <1> {1, 17}
Edge <(1, 17)>  




In [12]:
dset = DisjointSets()

for v in common_nodes:
    dset.make_set(v)

In [13]:
for seg in blue_segments:
    print(seg.portals, end=" -> ")
    for p in seg.portals:
        print(dset.find(p), end=" ")
    print(check_portals(seg.portals, dset))

{13, 23} -> 13 23 True
{5, 23} -> 5 23 True
{9, 5} -> 9 5 True
{1, 5} -> 1 5 True
{1, 17} -> 1 17 True


In [14]:
segments = list(red_segments)
segments.extend(blue_segments)

len(segments)

9

In [15]:
from random import shuffle
shuffle(segments)

In [16]:
child = UGraph()

for seg in segments:
    if check_portals(seg.portals, dset):
        print(f"Portal <{seg.portals}>")
        for edge in seg:
            v, u = edge
            print(edge, end="  ")
            child.add_edge(v, u)
        portals = iter(seg.portals)
        # Update dset 
        p1 = next(portals)
        for p in portals:
            dset.union(p1, p)
            p1 = p
        print("\n\n")


Portal <{5, 23}>
Edge <(5, 23)>  


Portal <{1, 17}>
Edge <(1, 17)>  


Portal <{1, 5}>
Edge <(14, 18)>  Edge <(5, 18)>  Edge <(1, 14)>  


Portal <{9, 5}>
Edge <(5, 9)>  


Portal <{13, 5}>
Edge <(11, 13)>  Edge <(5, 11)>  




In [17]:
has_cycle(child)

False

In [18]:
child.edges

defaultdict(set,
            {5: {9, 11, 18, 23},
             23: {5},
             1: {14, 17},
             17: {1},
             14: {1, 18},
             18: {5, 14},
             9: {5},
             11: {5, 13},
             13: {11}})

In [19]:
how_many_components(child)

1

In [20]:
for seg in blue_segments:
    print(seg.portals, end=" -> ")
    for p in seg.portals:
        print(dset.find(p), end=" ")
    print(check_portals(seg.portals, dset))

{13, 23} -> 23 23 False
{5, 23} -> 23 23 False
{9, 5} -> 23 23 False
{1, 5} -> 23 23 False
{1, 17} -> 23 23 False


## Crossing two random tree generated by Prim Random Steneir Tree

In [21]:
generator = GenerateBasedPrimRST(STPG)

red  = generator()
blue = generator()

In [22]:
red.edges == blue.edges

False

In [23]:
has_cycle(red), how_many_components(red)

(False, 1)

In [24]:
has_cycle(blue), how_many_components(blue)

(False, 1)

In [25]:
child     = UGraph()
red_only  = UGraph()
blue_only = UGraph()

In [26]:
for v, u in red.gen_undirect_edges():
    if blue.has_edge(v, u):
        child.add_edge(v, u)
    else:
        red_only.add_edge(v, u)

In [27]:
for v, u in blue.gen_undirect_edges():
    if not red.has_edge(v, u):
        blue_only.add_edge(v, u)

In [28]:
has_cycle(red_only), how_many_components(red_only)

(False, 5)

In [29]:
has_cycle(blue_only), how_many_components(blue_only)

(False, 9)

In [30]:
has_cycle(child), how_many_components(child)

(False, 9)

In [31]:
# 
red_leaves  = { v for v in red.vertices if red.degree(v) == 1}
blue_leaves = { v for v in blue.vertices if blue.degree(v) == 1}

common_nodes = set(red_only.vertices) & set(blue_only.vertices)

terminals = STPG.terminals.copy()

common_nodes & terminals

{16, 20, 23, 24, 31, 35, 37}

In [32]:
len(common_nodes)

20

In [33]:
red_segments  = find_segments(red_only, common_nodes)
blue_segments = find_segments(blue_only, common_nodes)

len(red_segments)

17

In [34]:
len(blue_segments)

15

In [35]:
queue = PriorityQueue()

for seg in red_segments:
    # print(seg, seg.cost)
    queue.push(seg.cost, seg)

for seg in blue_segments:
    # print(seg, seg.cost)
    queue.push(seg.cost, seg)

len(queue)

32

In [36]:
nodes = set(child.vertices)
print(len(nodes))

nodes.update(common_nodes)
print(len(nodes))

33
45


In [37]:
dset = DisjointSets()

nodes = set(child.vertices)
nodes.update(common_nodes)

for v in nodes:
    dset.make_set(v)

for v, u in child.gen_undirect_edges():
    dset.union(v, u)

In [38]:
len(dset)

45

In [39]:
dset.get_disjoint_sets()

defaultdict(set,
            {1: {1},
             2: {2, 12, 15, 28},
             14: {3, 14, 25, 30},
             49: {5, 11, 23, 34, 42, 44, 49},
             17: {6, 17, 21, 50},
             36: {7, 36, 38, 46},
             8: {8, 45},
             9: {9},
             40: {10, 13, 19, 40},
             16: {16},
             18: {18},
             39: {20, 39},
             22: {22, 47},
             24: {24},
             27: {27},
             31: {31},
             32: {32},
             35: {35},
             37: {37},
             43: {43},
             48: {48}})

In [40]:
while queue:
    seg = queue.pop()
    
    if check_portals(seg.portals, dset):
        # print('adding segment', seg)
        print(seg)
        for edge in seg:
            print(edge, end="  ")
            v, u = edge
            child.add_edge(v, u)
        
        portals = iter(seg.portals)
        # Update dset 
        print("")
        p_last = next(portals)
        print("Portals: ", p_last, end=" ")
        for p in portals:
            print(p, end=" ")
            dset.union(p_last, p)
            p_last = p
        print("\n\n")
    # else:
    #     print("####### >>> rejecting: ", seg)

Segment <1>
Edge <(25, 43)>  
Portals:  25 43 


Segment <1>
Edge <(46, 47)>  
Portals:  46 47 


Segment <1>
Edge <(2, 47)>  
Portals:  2 47 


Segment <1>
Edge <(23, 43)>  
Portals:  43 23 


Segment <2>
Edge <(35, 49)>  Edge <(20, 49)>  
Portals:  35 20 


Segment <1>
Edge <(1, 18)>  
Portals:  1 18 


Segment <1>
Edge <(34, 37)>  
Portals:  34 37 


Segment <1>
Edge <(16, 42)>  
Portals:  16 42 


Segment <1>
Edge <(27, 48)>  
Portals:  48 27 


Segment <1>
Edge <(9, 25)>  
Portals:  25 9 


Segment <1>
Edge <(42, 45)>  
Portals:  42 45 


Segment <2>
Edge <(10, 23)>  Edge <(10, 18)>  
Portals:  18 23 


Segment <1>
Edge <(14, 24)>  
Portals:  24 14 


Segment <1>
Edge <(11, 32)>  
Portals:  32 11 


Segment <1>
Edge <(23, 31)>  
Portals:  23 31 


Segment <1>
Edge <(15, 25)>  
Portals:  25 15 


Segment <1>
Edge <(25, 48)>  
Portals:  48 25 


Segment <1>
Edge <(1, 35)>  
Portals:  1 35 




In [41]:
for seg in red_segments:
    parent = dict()
    for p in seg.portals:
        if p in dset:
            parent[p] = dset.find(p)

    print(parent)

{48: 22, 25: 22}
{25: 22, 9: 22}
{25: 22, 43: 22}
{9: 22, 42: 22}
{9: 22, 37: 22}
{32: 22, 42: 22}
{42: 22, 45: 22}
{16: 22, 42: 22}
{24: 22, 45: 22}
{32: 22, 1: 22}
{2: 22, 47: 22}
{2: 22}
{27: 22, 12: 22, 46: 22}
{35: 22, 20: 22}
{23: 22, 31: 22}
{23: 22}
{18: 22, 23: 22}


In [42]:
for seg in blue_segments:
    parent = dict()
    for p in seg.portals:
        if p in dset:
            parent[p] = dset.find(p)
    print(parent)

{48: 22, 27: 22}
{48: 22, 25: 22, 18: 22}
{25: 22, 15: 22}
{1: 22, 18: 22}
{1: 22, 35: 22}
{46: 22, 47: 22}
{42: 22, 46: 22, 31: 22}
{31: 22}
{9: 22, 45: 22}
{43: 22, 23: 22}
{34: 22, 37: 22}
{32: 22, 11: 22}
{24: 22, 14: 22}
{20: 22, 44: 22}
{16: 22, 2: 22}


In [43]:
has_cycle(child)

True

In [44]:
how_many_components(child)

2

In [45]:
for seg in blue_segments:
    print(seg.portals)
    for edge in seg:
        print(edge, end='  ')
    print("")

{48, 27}
Edge <(27, 48)>  
{48, 25, 18}
Edge <(5, 25)>  Edge <(5, 18)>  Edge <(5, 48)>  
{25, 15}
Edge <(15, 25)>  
{1, 18}
Edge <(1, 18)>  
{1, 35}
Edge <(1, 35)>  
{46, 47}
Edge <(46, 47)>  
{42, 46, 31}
Edge <(26, 31)>  Edge <(6, 26)>  Edge <(6, 42)>  Edge <(6, 46)>  
{29, 31}
Edge <(29, 31)>  
{9, 45}
Edge <(36, 45)>  Edge <(9, 36)>  
{43, 23}
Edge <(23, 43)>  
{34, 37}
Edge <(34, 37)>  
{32, 11}
Edge <(11, 32)>  
{24, 14}
Edge <(14, 24)>  
{20, 44}
Edge <(19, 44)>  Edge <(19, 20)>  
{16, 2}
Edge <(2, 16)>  


In [46]:
for seg in red_segments:
    print(seg.portals)
    for edge in seg:
        print(edge, end='  ')
    print("")

{48, 25}
Edge <(25, 48)>  
{25, 9}
Edge <(9, 25)>  
{25, 43}
Edge <(25, 43)>  
{9, 42}
Edge <(9, 42)>  
{9, 37}
Edge <(9, 21)>  Edge <(21, 37)>  
{32, 42}
Edge <(32, 42)>  
{42, 45}
Edge <(42, 45)>  
{16, 42}
Edge <(16, 42)>  
{24, 45}
Edge <(24, 45)>  
{32, 1}
Edge <(1, 32)>  
{2, 47}
Edge <(2, 47)>  
{41, 2}
Edge <(4, 41)>  Edge <(2, 4)>  
{27, 12, 46}
Edge <(17, 27)>  Edge <(17, 46)>  Edge <(12, 17)>  
{35, 20}
Edge <(35, 49)>  Edge <(20, 49)>  
{23, 31}
Edge <(23, 31)>  
{33, 23}
Edge <(23, 33)>  
{18, 23}
Edge <(10, 23)>  Edge <(10, 18)>  


## Conclusão

Identifica os segmentos de um grafo, mas na reconstrução